In [101]:
import io
import numpy as np
import os
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
from scipy import stats
import sys


In [6]:
data_dir = '/home/share/Downloads'
confirmed_df = pd.read_csv(os.path.join(data_dir, 'covid_confirmed_usafacts.csv'))
deaths_df = pd.read_csv(os.path.join(data_dir, 'covid_deaths_usafacts.csv'))
pop_df = pd.read_csv(os.path.join(data_dir, 'covid_county_population_usafacts.csv'))

In [7]:
confirmed_df.head()

,countyFIPS,County Name,State,stateFIPS,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20,6/12/20,6/13/20
0,0,Statewide Unallocated,AL,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,241,248,259,265,272,282,295,312,323,331
2,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,293,296,304,313,320,325,331,343,353,361
3,1005,Barbour County,AL,1,0,0,0,0,0,0,...,177,183,190,193,197,199,208,214,221,226
4,1007,Bibb County,AL,1,0,0,0,0,0,0,...,76,76,77,77,79,85,89,93,97,100


In [8]:
dates = confirmed_df.columns[4:]
dates[0], dates[-1]

('1/22/20', '6/13/20')

In [9]:
confirmed_df_long = confirmed_df.melt(
    id_vars=['countyFIPS', 'County Name', 'State', 'stateFIPS'], 
    value_vars=dates, 
    var_name='Date', 
    value_name='Confirmed'
)

In [16]:
deaths_df_long = deaths_df.melt(
    id_vars=['countyFIPS', 'County Name', 'State', 'stateFIPS'], 
    value_vars=dates, 
    var_name='Date', 
    value_name='Deaths'
)

In [17]:
deaths_df_long.head(4)

,countyFIPS,County Name,State,stateFIPS,Date,Deaths
0,0,Statewide Unallocated,AL,1,1/22/20,0
1,1001,Autauga County,AL,1,1/22/20,0
2,1003,Baldwin County,AL,1,1/22/20,0
3,1005,Barbour County,AL,1,1/22/20,0


In [43]:
full_table = confirmed_df_long.merge(
  right=deaths_df_long, 
  how='left',
  on=['countyFIPS', 'County Name', 'State', 'stateFIPS', 'Date']
)


In [44]:
full_table.head(4)

,countyFIPS,County Name,State,stateFIPS,Date,Confirmed,Deaths
0,0,Statewide Unallocated,AL,1,1/22/20,0,0.0
1,1001,Autauga County,AL,1,1/22/20,0,0.0
2,1003,Baldwin County,AL,1,1/22/20,0,0.0
3,1005,Barbour County,AL,1,1/22/20,0,0.0


In [45]:
pop_df.head(4)

,countyFIPS,County Name,State,population
0,0,Statewide Unallocated,AL,0
1,1001,Autauga County,AL,55869
2,1003,Baldwin County,AL,223234
3,1005,Barbour County,AL,24686


In [46]:
pop_df[['countyFIPS','population']].head(4)

,countyFIPS,population
0,0,0
1,1001,55869
2,1003,223234
3,1005,24686


In [47]:
def get_pop(countyFIPS):
    return pop_df[pop_df['countyFIPS']==countyFIPS]
#
get_pop(1001)

,countyFIPS,County Name,State,population
1,1001,Autauga County,AL,55869


In [48]:
full_table = full_table[full_table['County Name']!='Statewide Unallocated'].reindex()

In [49]:
full_pop_df = full_table.join(pop_df.set_index('countyFIPS'), on='countyFIPS', rsuffix='_x')

In [50]:
full_pop_df[full_pop_df['County Name']!='Statewide Unallocated'].head(4)

,countyFIPS,County Name,State,stateFIPS,Date,Confirmed,Deaths,County Name_x,State_x,population
1,1001,Autauga County,AL,1,1/22/20,0,0.0,Autauga County,AL,55869.0
2,1003,Baldwin County,AL,1,1/22/20,0,0.0,Baldwin County,AL,223234.0
3,1005,Barbour County,AL,1,1/22/20,0,0.0,Barbour County,AL,24686.0
4,1007,Bibb County,AL,1,1/22/20,0,0.0,Bibb County,AL,22394.0


In [51]:
full_pop_df[full_pop_df['Confirmed']>298].count()

countyFIPS       30179
County Name      30179
State            30179
stateFIPS        30179
Date             30179
Confirmed        30179
Deaths           29791
County Name_x    30179
State_x          30179
population       30179
dtype: int64

In [66]:
full_significant = full_pop_df[ (full_pop_df['Confirmed']>298) & (full_pop_df['Deaths']>1) ].copy()
full_significant.count()


countyFIPS       29073
County Name      29073
State            29073
stateFIPS        29073
Date             29073
Confirmed        29073
Deaths           29073
County Name_x    29073
State_x          29073
population       29073
dtype: int64

In [67]:
full_significant['Death_per_Conf'] = full_significant['Deaths']/full_significant['Confirmed']

In [68]:
min(full_significant['Death_per_Conf']), max(full_significant['Death_per_Conf'])

(0.0011738466956215518, 0.19794871794871796)

In [70]:
dpc = np.array(full_significant['Death_per_Conf'])

In [73]:
np.average(dpc), np.median(dpc), np.std(dpc)

(0.04565294799715617, 0.04061895551257253, 0.029205118621984365)

In [74]:
dpc_ = (dpc - np.mean(dpc)) / np.std(dpc)

In [75]:
np.std(dpc_)

1.0

In [105]:
y = sorted(dpc)
x = range(len(y))
fig = px.scatter(x=x, y=y)
fig.update_layout(
    xaxis_title="county-index",
    yaxis_title="death to confirmed ratio"
)
fig.show()


In [81]:
max_ratio = max(dpc)
bucket_breaks = [max_ratio * i / 11 for i in range(12)]


In [87]:
bucket_counts = np.zeros(12)
bucket_sum = np.zeros(12)
for i in range(len(dpc)):
    bucket_idx = int((dpc[i] / (max_ratio * 1.0000001)) *12 )
    bucket_counts[bucket_idx] = bucket_counts[bucket_idx] + 1
    bucket_sum[bucket_idx] = bucket_sum[bucket_idx] + dpc[i]
    


In [104]:
xx = [bucket_sum[i]/bucket_counts[i] for i in range(12)]
fig = px.scatter(x=xx, y=bucket_counts)
fig.update_layout(
    xaxis_title="death to confirmed ratio",
    yaxis_title="county-count"
)

# Mock what the normal dist SHOULD look like
samples = np.random.normal(size=10000)
bins = np.linspace(-5, 5, 30)
histogram, bins = np.histogram(samples, bins=bins, density=True)
bin_centers = np.array(0.5*(bins[1:] + bins[:-1]))
pdf = stats.norm.pdf(bin_centers) 
pdf = pdf * 20000
bin_centers = bin_centers / 5000 + .002

fig.add_trace(go.Scatter(x=bin_centers, y=pdf))
fig.show()
